In [48]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

In [5]:
df_1t = pd.read_csv('./data/review_data_1t_2.csv', encoding='utf-8')
df_1t.head(2)

,label,review,review2,adj,verb,noun,exclamation
0,희양산,[2일차: 희양산 - 대야산4山_희양산 50m© NAVER Corp.더보기 /Ope...,일차 희양산 대야산희양산 더보기 지도 데이터 지도 컨트롤러 범례부동산거리읍면동시군구...,간단한 시원한 괜찮다 어렵고 길도 험하고 불친절한 그렇지 나쁘지 많은 미끄럽 없는 ...,둘러 삼아 바라본 하면 나오는 찾아 찾아보니 앞서던 넘치고 않았던 않다 거칠어진 기...,일차 희양산 대야산 희양산 더 보기 지도 데이터 지도 컨트롤러 범례 부동산 거리 읍...,휴
1,희양산,[몇년전 부봉인가 도명산을 올랐을때 유난히 대머리가 심한 희양산이 눈에 띄어 언젠가...,몇년전 부봉인가 도명산을 올랐을때 유난히 대머리가 심한 희양산이 눈에 띄어 언젠가 ...,심한 뿌옇다 맑아진다니 간략히 희 많지 짧게 덥고 모호한데 예쁘다고 그래 있는 있 ...,올랐을 띄어 가보고 싶었는데 나면 트여 볼거란 들었다 헌데 가르는 기대는 버리지 세...,년전 부 봉인 도명 산 때 대머리 희양산 눈 한번 오늘이 그날 토요일 새벽 비 그 ...,끄응 아 엉


In [3]:
df_2t = pd.read_csv('./data/review_data_2t_2.csv', encoding='utf-8')
df_2t.head(2)

,review,label,review2,adj,verb,noun,exclamation
0,?,문화,NaN,NaN,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,NaN,NaN,NaN


In [6]:
df_3t = pd.read_csv('./data/review_data_3t_2.csv', encoding='utf-8')
df_3t.head(2)

,리뷰,음식유형,review2,adj,verb,noun,exclamation
0,👍,디저트,NaN,NaN,NaN,NaN,NaN
1,굿,디저트,굿,NaN,NaN,굿,NaN


In [8]:
df_1t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123983 entries, 0 to 123982
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   label        123983 non-null  object
 1   review       123983 non-null  object
 2   review2      123983 non-null  object
 3   adj          121758 non-null  object
 4   verb         122764 non-null  object
 5   noun         123833 non-null  object
 6   exclamation  30626 non-null   object
dtypes: object(7)
memory usage: 6.6+ MB


In [7]:
df_2t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141254 entries, 0 to 141253
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review       141254 non-null  object
 1   label        141254 non-null  object
 2   review2      136766 non-null  object
 3   adj          101343 non-null  object
 4   verb         59979 non-null   object
 5   noun         93857 non-null   object
 6   exclamation  1212 non-null    object
dtypes: object(7)
memory usage: 7.5+ MB


In [9]:
df_3t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123749 entries, 0 to 123748
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   리뷰           117078 non-null  object
 1   음식유형         123749 non-null  object
 2   review2      115595 non-null  object
 3   adj          95611 non-null   object
 4   verb         64770 non-null   object
 5   noun         97115 non-null   object
 6   exclamation  1270 non-null    object
dtypes: object(7)
memory usage: 6.6+ MB


In [15]:
# 중복제거전
len(' '.join(df_2t['adj'].dropna().sort_values()).split())

235178

In [29]:
# 중복제거후
adj_list_1t = ' '.join(df_1t['adj'].dropna().sort_values().drop_duplicates()).split()
adj_list_1t = list(set(adj_list_1t))
print(len(adj_list_1t))

114700


In [28]:
# 중복제거후
adj_list_2t = ' '.join(df_2t['adj'].dropna().sort_values().drop_duplicates()).split()
adj_list_2t = list(set(adj_list_2t))
print(len(adj_list_2t))

12912


In [30]:
# 중복제거후
adj_list_3t = ' '.join(df_3t['adj'].dropna().sort_values().drop_duplicates()).split()
adj_list_3t = list(set(adj_list_3t))
print(len(adj_list_3t))

11990


In [35]:
tmp=pd.concat([pd.Series(adj_list_2t),pd.Series(adj_list_3t)]).drop_duplicates()
len(tmp)

19408

In [37]:
tmp=pd.concat([pd.Series(adj_list_1t),pd.Series(adj_list_2t),pd.Series(adj_list_3t)]).drop_duplicates()
len(tmp)

120042

In [49]:
# 어간 추출
def extract_stem(words):
    okt = Okt()
    stems = []
    for word in words:
        # 어간 추출
        stems = okt.normalize(words)
    return stems

In [53]:
from tqdm import tqdm
tqdm.pandas()
adj_norm_list_2t = pd.Series(adj_list_2t).progress_map(extract_stem)

100%|███████████████████████████████████████████████████████████████████████████| 12912/12912 [00:28<00:00, 446.91it/s]


In [54]:
len(adj_norm_list_2t)

12912

In [55]:
from tqdm import tqdm
tqdm.pandas()
adj_norm_list_3t = pd.Series(adj_list_3t).progress_map(extract_stem)

100%|███████████████████████████████████████████████████████████████████████████| 11990/11990 [00:26<00:00, 454.47it/s]
